# Try something

In [156]:
import numpy as np
keys = ['a', 'b', 'c', 'd', 'e', 'f']
items = np.random.rand(6)

In [157]:
x = np.random.rand(1,10,100)

In [158]:
col = [2, 3, 5]

In [159]:
x.shape

(1, 10, 100)

In [160]:
x[:, col].shape

(1, 3, 100)

In [161]:
data = dict(zip(keys,items))

In [162]:
d = ['a', 'b']

In [163]:
%%timeit
target_index = [keys.index(i) for i in d if i in keys]

KeyboardInterrupt: 

In [ ]:
%%timeit
[i for i in d if i in keys]

171 ns ± 4.37 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [ ]:
%%timeit
set(d).intersection(keys)

186 ns ± 2.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [ ]:
%%timeit
[set(d) & set(keys)]

278 ns ± 19.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Root with torch

In [ ]:
import os
data_path = '/home/northnpk/Downloads/JetClass_dataset/'
train_path = [data_path + 'train/' + p for p in os.listdir(data_path + 'train')]
val_path = [data_path + 'val/' + p for p in os.listdir(data_path + 'val')]
test_path = [data_path + 'test/' + p for p in os.listdir(data_path + 'test')]
print(f'Train path: {train_path}')
print(f'Validation path: {val_path}')
print(f'Test path: {test_path}')

Train path: ['/home/northnpk/Downloads/JetClass_dataset/train/TTBar_067.root', '/home/northnpk/Downloads/JetClass_dataset/train/HToWW4Q_051.root', '/home/northnpk/Downloads/JetClass_dataset/train/HToBB_079.root', '/home/northnpk/Downloads/JetClass_dataset/train/HToWW4Q_021.root', '/home/northnpk/Downloads/JetClass_dataset/train/TTBarLep_054.root', '/home/northnpk/Downloads/JetClass_dataset/train/HToBB_097.root', '/home/northnpk/Downloads/JetClass_dataset/train/TTBar_024.root', '/home/northnpk/Downloads/JetClass_dataset/train/HToWW2Q1L_089.root', '/home/northnpk/Downloads/JetClass_dataset/train/TTBarLep_078.root', '/home/northnpk/Downloads/JetClass_dataset/train/HToCC_035.root', '/home/northnpk/Downloads/JetClass_dataset/train/HToCC_092.root', '/home/northnpk/Downloads/JetClass_dataset/train/ZJetsToNuNu_006.root', '/home/northnpk/Downloads/JetClass_dataset/train/TTBar_015.root', '/home/northnpk/Downloads/JetClass_dataset/train/HToCC_077.root', '/home/northnpk/Downloads/JetClass_dataset/

In [ ]:
import torch
import ROOT

tree_name = "tree"
# file_name = "/home/northnpk/Downloads/test_20M/HToCC_100.root"
# tree_name = "sig_tree"
# file_name = "http://root.cern/files/Higgs_data.root"
batch_size = 128
chunk_size = 5_000
vec_columns=['part_px', 'part_py', 'part_pz', 'part_energy']
columns=['jet_pt', 'jet_eta', 'jet_phi', 'jet_energy']
# columns.extend(vec_columns)
max_vec_sizes = dict(zip(vec_columns, [128]*4))
# target = 'Type'
# targets = 'label_Hcc'
# targets = ['label_QCD', 'label_Hbb', 'label_Hcc']
targets = ['label_QCD', 'label_Hbb', 'label_Hcc', 'label_Hgg', 'label_H4q',
                'label_Hqql', 'label_Zqq', 'label_Wqq', 'label_Tbqq', 'label_Tbl']

In [ ]:
# Returns two generators that return training and validation batches
# as PyTorch tensors.
gen_train, _ = ROOT.TMVA.Experimental.CreatePyTorchGenerators(
    tree_name,
    train_path,
    batch_size,
    chunk_size,
    columns=columns+vec_columns+targets,
    max_vec_sizes=max_vec_sizes,
    targets=targets,
    validation_split=0,
)

In [ ]:
for i, (d, y) in enumerate(gen_train):
    print(d)

tensor([[ 5.3210e+02,  2.9675e-02, -2.6256e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 5.0854e+02, -1.1659e+00, -4.2197e-01,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 5.0222e+02,  9.2839e-01, -2.1483e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 5.2360e+02,  5.3159e-01,  2.4067e-01,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 5.2833e+02, -9.5574e-01,  2.6223e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 9.7276e+02, -1.3443e+00, -7.6502e-01,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])
tensor([[ 5.6325e+02, -6.5745e-01,  2.8081e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 5.6509e+02, -3.9772e-01, -2.3216e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 6.1960e+02, -9.4892e-01, -2.1425e-01,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 6.2102e+02,  1.2954e+00, -2.8123e-01

In [ ]:
# Get a list of the columns used for training
input_columns = gen_train.train_columns
num_features = len(input_columns)
num_targets = len(gen_train.target_columns)

In [ ]:
# input_columns

In [ ]:
len(gen_train.target_columns)

10

In [ ]:
def calc_accuracy(targets, pred):
    return torch.sum(targets == pred.round()) / pred.size(0) / num_targets
 
 
# Initialize PyTorch model
model = torch.nn.Sequential(
    torch.nn.Linear(num_features, 300),
    torch.nn.Tanh(),
    torch.nn.Linear(300, 300),
    torch.nn.Tanh(),
    torch.nn.Linear(300, 300),
    torch.nn.Tanh(),
    torch.nn.Linear(300, num_targets),
    torch.nn.Sigmoid(),
)
loss_fn = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
 

In [ ]:
 # Loop through the training set and train model
for i, (x_train, y_train) in enumerate(gen_train):
    # Make prediction and calculate loss
    pred = model(x_train)
    # print(pred.to_numpy)
    loss = loss_fn(pred, y_train)
 
    # improve model
    model.zero_grad()
    loss.backward()
    optimizer.step()
 
    # Calculate accuracy
    # accuracy = calc_accuracy(y_train, pred)
 
    print(f"Training => loss: {loss}")

Training => loss: 0.10594888031482697
Training => loss: 0.10407759994268417
Training => loss: 0.1027073860168457
Training => loss: 0.10169653594493866
Training => loss: 0.10088904201984406
Training => loss: 0.10030145943164825
Training => loss: 0.09983847290277481
Training => loss: 0.0994872897863388
Training => loss: 0.09917252510786057
Training => loss: 0.09895814955234528
Training => loss: 0.09876831620931625
Training => loss: 0.09861672669649124
Training => loss: 0.09849604219198227
Training => loss: 0.09836702048778534
Training => loss: 0.09825899451971054
Training => loss: 0.09817083179950714
Training => loss: 0.09808899462223053
Training => loss: 0.09801939874887466
Training => loss: 0.097938172519207
Training => loss: 0.0978364497423172
Training => loss: 0.09779436141252518
Training => loss: 0.097709059715271
Training => loss: 0.09765318036079407
Training => loss: 0.09757114946842194
Training => loss: 0.09749530255794525
Training => loss: 0.0974443331360817
Training => loss: 0.

In [ ]:
import sys
sys.getsizeof(gen_train)

In [ ]:
#################################################################
# Validation
#################################################################
 
# Evaluate the model on the validation set
for i, (x_train, y_train) in enumerate(gen_validation):
    # Make prediction and calculate accuracy
    pred = model(x_train)
    accuracy = calc_accuracy(y_train, pred)
 
    print(f"Validation => accuracy: {accuracy}")

Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accura

# Load with uproot and Jetclass github dataloader codes

In [ ]:
!pip install vector

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.6/182.6 KB 3.3 MB/s eta 0:00:00a 0:00:01


In [ ]:
import numpy as np
import awkward as ak
import uproot
import vector
vector.register_awkward()


def read_file(
        filepath,
        max_num_particles=128,
        particle_features=['part_pt', 'part_eta', 'part_phi', 'part_energy'],
        jet_features=['jet_pt', 'jet_eta', 'jet_phi', 'jet_energy'],
        labels=['label_QCD', 'label_Hbb', 'label_Hcc', 'label_Hgg', 'label_H4q',
                'label_Hqql', 'label_Zqq', 'label_Wqq', 'label_Tbqq', 'label_Tbl']):
    """Loads a single file from the JetClass dataset.

    **Arguments**

    - **filepath** : _str_
        - Path to the ROOT data file.
    - **max_num_particles** : _int_
        - The maximum number of particles to load for each jet. 
        Jets with fewer particles will be zero-padded, 
        and jets with more particles will be truncated.
    - **particle_features** : _List[str]_
        - A list of particle-level features to be loaded. 
        The available particle-level features are:
            - part_px
            - part_py
            - part_pz
            - part_energy
            - part_pt
            - part_eta
            - part_phi
            - part_deta: np.where(jet_eta>0, part_eta-jet_p4, -(part_eta-jet_p4))
            - part_dphi: delta_phi(part_phi, jet_phi)
            - part_d0val
            - part_d0err
            - part_dzval
            - part_dzerr
            - part_charge
            - part_isChargedHadron
            - part_isNeutralHadron
            - part_isPhoton
            - part_isElectron
            - part_isMuon
    - **jet_features** : _List[str]_
        - A list of jet-level features to be loaded. 
        The available jet-level features are:
            - jet_pt
            - jet_eta
            - jet_phi
            - jet_energy
            - jet_nparticles
            - jet_sdmass
            - jet_tau1
            - jet_tau2
            - jet_tau3
            - jet_tau4
    - **labels** : _List[str]_
        - A list of truth labels to be loaded. 
        The available label names are:
            - label_QCD
            - label_Hbb
            - label_Hcc
            - label_Hgg
            - label_H4q
            - label_Hqql
            - label_Zqq
            - label_Wqq
            - label_Tbqq
            - label_Tbl

    **Returns**

    - x_particles(_3-d numpy.ndarray_), x_jets(_2-d numpy.ndarray_), y(_2-d numpy.ndarray_)
        - `x_particles`: a zero-padded numpy array of particle-level features 
                         in the shape `(num_jets, num_particle_features, max_num_particles)`.
        - `x_jets`: a numpy array of jet-level features
                    in the shape `(num_jets, num_jet_features)`.
        - `y`: a one-hot encoded numpy array of the truth lables
               in the shape `(num_jets, num_classes)`.
    """

    def _pad(a, maxlen, value=0, dtype='float32'):
        if isinstance(a, np.ndarray) and a.ndim >= 2 and a.shape[1] == maxlen:
            return a
        elif isinstance(a, ak.Array):
            if a.ndim == 1:
                a = ak.unflatten(a, 1)
            a = ak.fill_none(ak.pad_none(a, maxlen, clip=True), value)
            return ak.values_astype(a, dtype)
        else:
            x = (np.ones((len(a), maxlen)) * value).astype(dtype)
            for idx, s in enumerate(a):
                if not len(s):
                    continue
                trunc = s[:maxlen].astype(dtype)
                x[idx, :len(trunc)] = trunc
            return x

    table = uproot.open(filepath)['tree'].arrays()
    p4 = vector.zip({'px': table['part_px'],
                     'py': table['part_py'],
                     'pz': table['part_pz'],
                     'energy': table['part_energy']})
    table['part_pt'] = p4.pt
    table['part_eta'] = p4.eta
    table['part_phi'] = p4.phi

    x_particles = np.stack([ak.to_numpy(_pad(table[n], maxlen=max_num_particles)) for n in particle_features], axis=1)
    x_particles = x_particles.reshape(x_particles.shape(0), x_particles.shape(1)*x_particles.shape(2))
    x_jets = np.stack([ak.to_numpy(table[n]).astype('float32') for n in jet_features], axis=1)
    X = np.stack([x_particles, x_jets], axis=1)
    y = np.stack([ak.to_numpy(table[n]).astype('int') for n in labels], axis=1)

    return X, y

In [ ]:
# table = uproot.open("/home/northnpk/Downloads/test_20M/HToCC_100.root")['tree']

In [ ]:
# table.arrays()[0]

In [ ]:
# higgs = uproot.open("/home/northnpk/Higgs_data.root")['sig_tree'].arrays()

In [ ]:
# higgs[0]

In [ ]:
X, y = read_file("/home/northnpk/Downloads/test_20M/HToCC_100.root")

FileNotFoundError: [Errno 2] No such file or directory: '/home/northnpk/Downloads/test_20M/HToCC_100.root'

In [ ]:
x_pt.shape

(100000, 4, 128)

In [ ]:
x_jet.shape

(100000, 4)

In [ ]:
y.shape

(100000, 10)

# C++ test multiple file loader

In [ ]:
import torch
import ROOT

tree_name = "tree"
# file_name = ["/home/northnpk/Downloads/test_20M/HToCC_100.root", "/home/northnpk/Downloads/test_20M/HToCC_101.root"]
# file_name = "/home/northnpk/Downloads/test_20M/HToCC_100.root"
import os
data_path = '/home/northnpk/Downloads/JetClass_dataset/'
train_path = [data_path + 'train/' + p for p in os.listdir(data_path + 'train')]
val_path = [data_path + 'val/' + p for p in os.listdir(data_path + 'val')]
test_path = [data_path + 'test/' + p for p in os.listdir(data_path + 'test')]

# tree_name = "sig_tree"
# file_name = "http://root.cern/files/Higgs_data.root"
batch_size = 128
chunk_size = 5_000
vec_columns=['part_px', 'part_py', 'part_pz', 'part_energy']
columns=['jet_pt', 'jet_eta', 'jet_phi', 'jet_energy']
# columns.extend(vec_columns)
max_vec_sizes = dict(zip(vec_columns, [128]*4))
# target = 'Type'
# targets = 'label_Hcc'
# targets = ['label_QCD', 'label_Hbb', 'label_Hcc']
targets = ['label_QCD', 'label_Hbb', 'label_Hcc', 'label_Hgg', 'label_H4q',
                'label_Hqql', 'label_Zqq', 'label_Wqq', 'label_Tbqq', 'label_Tbl']

In [ ]:
%%cpp
#include "TSystem.h"

double check_mem(std::string s = ""){

   ProcInfo_t p;
   printf("%s - ",s.c_str());
   gSystem->GetProcInfo(&p);
   printf(" Rmem = %8.3f MB, Vmem = %8.f3 MB  \n",
          p.fMemResident * 1e-3,  /// Real memory to watch for leaks                                                                                                                                                                                      
          p.fMemVirtual  * 1e-3
      );
   return p.fMemResident * 1e-3;
}

In [ ]:

# Returns two generators that return training and validation batches
# as PyTorch tensors.
gen_train, _ = ROOT.TMVA.Experimental.CreatePyTorchGenerators(
    tree_name,
    train_path,
    batch_size,
    chunk_size,
    columns=columns+vec_columns+targets,
    max_vec_sizes=max_vec_sizes,
    targets=targets,
    validation_split=0,
)

TypeError: CreatePyTorchGenerators() got multiple values for argument 'columns'

In [ ]:
count = 0
for i, (d, y) in enumerate(gen_train):
    check_x, check_y = d, y
    mem = ROOT.check_mem("gen_train")
    count+=1
print(f'Batch count: {count}')
print(f'Net mem: {mem}')

NameError: name 'gen_train' is not defined

In [ ]:
# Get a list of the columns used for training
input_columns = gen_train.train_columns
num_features = len(input_columns)
num_targets = len(gen_train.target_columns)

In [ ]:
num_features

516

In [ ]:
len(gen_train.target_columns)

10

In [ ]:
def calc_accuracy(targets, pred):
    return torch.sum(targets == pred.round()) / pred.size(0) / num_targets
 
 
# Initialize PyTorch model
model = torch.nn.Sequential(
    torch.nn.Linear(num_features, 300),
    torch.nn.Tanh(),
    torch.nn.Linear(300, 300),
    torch.nn.Tanh(),
    torch.nn.Linear(300, 300),
    torch.nn.Tanh(),
    torch.nn.Linear(300, num_targets),
    torch.nn.Sigmoid(),
)
loss_fn = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [ ]:
 # Loop through the training set and train model
for i, (x_train, y_train) in enumerate(gen_train):
    # Make prediction and calculate loss
    pred = model(x_train)
    # print(pred.to_numpy)
    loss = loss_fn(pred, y_train)
 
    # improve model
    model.zero_grad()
    loss.backward()
    optimizer.step()
 
    # Calculate accuracy
    accuracy = calc_accuracy(y_train, pred)
 
    print(f"Training => accuracy: {accuracy}")

Training => accuracy: 0.675000011920929
Training => accuracy: 0.7289062738418579
Training => accuracy: 0.840624988079071
Training => accuracy: 0.90234375
Training => accuracy: 0.9429687261581421
Training => accuracy: 0.987500011920929
Training => accuracy: 0.999218761920929
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1.0
Training => accuracy: 1

In [ ]:
#################################################################
# Validation
#################################################################
 
# Evaluate the model on the validation set
for i, (x_train, y_train) in enumerate(gen_validation):
    # Make prediction and calculate accuracy
    pred = model(x_train)
    accuracy = calc_accuracy(y_train, pred)
 
    print(f"Validation => accuracy: {accuracy}")

Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accuracy: 1.0
Validation => accura

# load from RDataFrame

In [1]:
import ROOT
from ROOT import RDataFrame
# names = ROOT.std.vector('string')()

In [2]:
from tqdm.auto import tqdm

/home/northnpk/python-custom-build/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
data_path = '/home/northnpk/Downloads/JetClass_dataset/'
train_path = [data_path + 'train/' + p for p in os.listdir(data_path + 'train')]
# for n in train_path: names.push_back(n)

In [4]:
tree_name = "tree"
# tree_name = "sig_tree"
# file_name = "http://root.cern/files/Higgs_data.root"
batch_size = 128
chunk_size = 5_000
vec_columns=['part_px', 'part_py', 'part_pz', 'part_energy']
columns=['jet_pt', 'jet_eta', 'jet_phi', 'jet_energy']
# columns.extend(vec_columns)
max_vec_sizes = dict(zip(vec_columns, [128]*4))
# target = 'Type'
# targets = 'label_Hcc'
# targets = ['label_QCD', 'label_Hbb', 'label_Hcc']
targets = ['label_QCD', 'label_Hbb', 'label_Hcc', 'label_Hgg', 'label_H4q',
                'label_Hqql', 'label_Zqq', 'label_Wqq', 'label_Tbqq', 'label_Tbl']

In [5]:
# r_df = RDataFrame(tree_name, names[:10])
r_df = RDataFrame(tree_name, train_path[:10])

In [6]:
r_df.GetColumnNames()

vector<string>{ "aux_genpart_eta", "aux_genpart_phi", "aux_genpart_pid", "aux_genpart_pt", "aux_truth_match", "jet_energy", "jet_eta", "jet_nparticles", "jet_phi", "jet_pt", "jet_sdmass", "jet_tau1", "jet_tau2", "jet_tau3", "jet_tau4", "label_H4q", "label_Hbb", "label_Hcc", "label_Hgg", "label_Hqql", "label_QCD", "label_Tbl", "label_Tbqq", "label_Wqq", "label_Zqq", "part_charge", "part_d0err", "part_d0val", "part_deta", "part_dphi", "part_dzerr", "part_dzval", "part_energy", "part_isChargedHadron", "part_isElectron", "part_isMuon", "part_isNeutralHadron", "part_isPhoton", "part_px", "part_py", "part_pz" }

In [ ]:
def split_columns(str):
    jet = []
    part = []
    label = []
    rest = []
    for name in 

In [7]:
# Returns two generators that return training and validation batches
# as PyTorch tensors.
gen_train = ROOT.TMVA.Experimental.CreatePyTorchGenerators(
    r_df,
    batch_size,
    chunk_size,
    columns=columns+vec_columns+targets,
    max_vec_sizes=max_vec_sizes,
    target=targets,
    validation_split=0,
    drop_remainder=False
)

In [8]:
from torch.utils.data import DataLoader, IterableDataset
import numpy as np

In [9]:
class RBatchDataset(IterableDataset):
    def __init__(self, generator, vector_prep_fn=None):
        self.generator = generator
        self.batch_size = self.generator.base_generator.batch_size
        self.vector_prep_fn = vector_prep_fn
        self.apply_vector_prep_fn = False
        if self.vector_prep_fn != None:
            self.apply_vector_prep_fn = True
        # print(f'Batch size: {self.batch_size}')
        if self.generator.last_batch_no_of_rows != 0 and self.generator.number_of_batches > 1:
            self.length = ((self.generator.number_of_batches-1) * self.batch_size) + self.generator.last_batch_no_of_rows
        elif self.generator.number_of_batches == 1:
            self.length = self.generator.last_batch_no_of_rows
        else :
            self.length = (gen_train.number_of_batches * self.batch_size)
        
        self.vec_columns = [c not in self.generator.base_generator.given_columns for c in self.generator.base_generator.train_columns]
        self._columns = [c in self.generator.base_generator.given_columns for c in self.generator.base_generator.train_columns]
        self.label_columns = self.generator.base_generator.target_columns
        self.vec_names = []
        for n in self.generator.base_generator.given_columns:
            if n not in self.label_columns and n not in self.generator.base_generator.train_columns:
                self.vec_names.append(n)
    
    def collate_fn(self, data):
        tensors, targets = data
        if self.apply_vector_prep_fn :
            return self.vector_prep_fn(tensors[:, self.vec_columns].reshape(len(tensors), 4, 128), self.vec_names), tensors[:, self._columns], targets
        
        else :
            return tensors[:, self.vec_columns].reshape(len(tensors), 4, 128), tensors[:, self._columns], targets
    
    def __iter__(self):
        return self.generator.__iter__()
        
    def __len__(self):
        return self.generator.number_of_batches

In [10]:
import vector
import torch
import awkward as ak

# zip(torch)->awkward->numpy->torch
def vector_prep_fn_1(tensors, vec_names):
    p4 = vector.zip({name.replace('part_', ''): tensors[:, idx] for idx, name in enumerate(vec_names)})
    return torch.stack((torch.from_numpy(ak.to_numpy(p4.pt)), torch.from_numpy(ak.to_numpy(p4.eta)), torch.from_numpy(ak.to_numpy(p4.phi)), tensors[:, -1]), dim=1)

# zip(torch)->awkward->torch
def vector_prep_fn_2(tensors, vec_names):
    p4 = vector.zip({name.replace('part_', ''): tensors[:, idx] for idx, name in enumerate(vec_names)})
    return torch.stack((torch.tensor(p4.pt), torch.tensor(p4.eta), torch.tensor(p4.phi), tensors[:, -1]), dim=1)

# torch->numpy->array(numpy)->numpy->torch
def vector_prep_fn_3(tensors, vec_names):
    tensors = tensors.numpy()
    p4 = vector.arr({name.replace('part_', ''): tensors[:, idx] for idx, name in enumerate(vec_names)})
    return torch.from_numpy(np.stack((p4.pt, p4.eta, p4.phi, tensors[:, -1]), axis=1))


In [11]:
# dataset = RBatchDataset(gen_train, vector_prep_fn=vector_prep_fn_3)
dataset = RBatchDataset(gen_train)
loader = DataLoader(dataset=dataset, collate_fn=dataset.collate_fn, batch_size=None)

In [12]:
# %%timeit
# for data in loader:
#     _ = data

In [13]:
import awkward as ak
def drop_pad(tensors):
    padding_value = 0.0000
    data_i = []
    for e in tensors:
        data_j = []
        for v in e:
            data_k = []
            for _ in v:
                if _ != padding_value:
                    data_k.append(_)
            data_j.append(data_k)
        data_i.append(data_j)
    return ak.to_regular(ak.Array(data_i))

In [14]:
def _pad(a, maxlen, value=0, dtype='float32'):
    if isinstance(a, np.ndarray) and a.ndim >= 2 and a.shape[1] == maxlen:
        return a
    elif isinstance(a, ak.Array):
        if a.ndim == 1:
            a = ak.unflatten(a, 1)
        a = ak.fill_none(ak.pad_none(a, maxlen, clip=True), value)
        return ak.values_astype(a, dtype)
    else:
        x = (np.ones((len(a), maxlen)) * value).astype(dtype)
        for idx, s in enumerate(a):
            if not len(s):
                continue
            trunc = s[:maxlen].astype(dtype)
            x[idx, :len(trunc)] = trunc
        return x

In [15]:
def vecterize_ak(tensors):
    p4 = vector.zip({'px': tensors[:, 0], 
              'py': tensors[:, 1],
              'pz': tensors[:, 2],
              'energy': tensors[:, 3]})
    return torch.from_numpy(np.stack([_pad(i, maxlen=128) for i in [p4.pt, p4.eta, p4.phi, tensors[:, 3]]], axis=1).to_numpy())
    

In [16]:
for data in loader:
    sample = data
    break

In [21]:
for i in sample:
    print(i.shape)

torch.Size([128, 4, 128])
torch.Size([128, 4])
torch.Size([128, 10])


In [17]:
sample_no_pad = drop_pad(sample[0])

In [18]:
%%timeit
vecterize_ak(sample_no_pad)

9.26 ms ± 233 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
# zip(torch)->awkward->numpy->torch
vector_prep_fn_1(sample[0], loader.dataset.vec_names)

5.63 ms ± 52 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%%timeit
# zip(torch)->awkward->torch
vector_prep_fn_2(sample[0], loader.dataset.vec_names)

561 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
# torch->numpy->array(numpy)->numpy->torch
vector_prep_fn_3(sample[0], loader.dataset.vec_names)

470 μs ± 10.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
cos = torch.nn.CosineSimilarity(dim=2, eps=1e-6)
cos(vecterize_ak(drop_pad(sample[0])), vector_prep_fn_3(sample[0], loader.dataset.vec_names))

tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1

In [287]:
cos(torch.tensor([[0.,0.]]), torch.tensor([[0.,0.]]))

tensor([0.])

In [104]:
%%cpp
#include "TSystem.h"

double check_mem(std::string s = ""){

   ProcInfo_t p;
   printf("%s - ",s.c_str());
   gSystem->GetProcInfo(&p);
   /*printf(" Rmem = %8.3f MB, Vmem = %8.f3 MB  \n",
          p.fMemResident * 1e-3,  /// Real memory to watch for leaks                                                                                                                                                                                      
          p.fMemVirtual  * 1e-3
      );*/
   return p.fMemResident * 1e-3;
}

In [12]:
%load_ext memray

In [13]:
def _iter():
    count = 0
    for i, (d, y) in enumerate(tqdm(gen_train)):
        check_x, check_y = d, y
        # mem = ROOT.check_mem("gen_train")
        count+=1
# print('On branch: copycopycopywithbuffer')
# print(f'Batch count: {count}')
# print(f'Net mem: {mem}')

In [14]:
%%timeit
_iter()

100%|██████████| 7813/7813 [00:26<00:00, 297.83it/s]

26.5 s ± 412 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
import torch
# Initialize PyTorch model
class SimpleJetClassModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.part_conv = torch.nn.Conv1d(4, 1, 3, stride=1, padding= "same")
        self.part_pool = torch.nn.AvgPool1d(2, 2)
        self.jet_liner = torch.nn.Linear(4, 4)
        self.drop1 = torch.nn.Dropout(0.5)
        self.drop2 = torch.nn.Dropout(0.5)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()
        self.flat = torch.nn.Flatten()
        self.fc = torch.nn.Linear(68, 10)
 
    def forward(self, x_part, x_jet):
        x_part = self.act1(self.part_conv(x_part))
        x_part = self.drop1(x_part)
        x_part = self.part_pool(x_part)
        x_part = self.flat(x_part)
        x_jet = self.act2(self.jet_liner(x_jet))
        x_jet = self.drop2(x_jet)
        return self.fc(torch.concat((x_part, x_jet), dim=1))
 
model = SimpleJetClassModel()

loss_fn = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

print(model)

print('Start training...')
 # Loop through the training set and train model

def train():
    sum_loss = 0
    for i, (x_train, y_train) in enumerate(tqdm(gen_train)):
        # Make prediction and calculate loss
        # tmp = (x_train, y_train)
        # print(tmp)
        pred = model(x_train)
        # print(pred.to_numpy)
        loss = loss_fn(pred, y_train.to(torch.float32))
    
        # improve model
        model.zero_grad()
        loss.backward()
        optimizer.step()
    
        # Calculate accuracy
        sum_loss += loss

    print(f"Training => loss: {sum_loss/1000000}")

CIFAR10Model(
  (conv1): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc3): Linear(in_features=672, out_features=512, bias=True)
  (act3): ReLU()
  (drop3): Dropout(p=0.5, inplace=False)
  (fc4): Linear(in_features=512, out_features=10, bias=True)
)
Start training...


In [15]:
import timeit
print(f"tmvaRBatchGenerator Training time avg.: {timeit.timeit('train()', number=3, globals=globals())/3.0}")
print('Finished training')
print('Ended')

100%|██████████| 7812/7812 [01:17<00:00, 100.44it/s]


Training => loss: nan


100%|██████████| 7812/7812 [01:09<00:00, 113.06it/s]


Training => loss: nan


 84%|████████▍ | 6594/7812 [01:01<00:11, 108.02it/s]


Training => loss: nan
tmvaRBatchGenerator Training time avg.: 69.358968877333
Finished training
Ended


In [8]:
%%memray_flamegraph --native
_iter()

# Try to load with akward and Uproot

In [1]:
import numpy as np
import awkward as ak
import ROOT
from ROOT import RDataFrame
import uproot
import vector
import torch
from torch.utils.data import DataLoader, TensorDataset
from concurrent.futures import ProcessPoolExecutor
vector.register_awkward()
from tqdm.auto import tqdm

def chunk_processing(file, _pad, max_num_particles, particle_features, jet_features, labels):
    chunk_X = []
    chunk_y = []
    for table in uproot.iterate(file, step_size=10000):
        # print('table from rdf')
        # print(table)
        x_particles = np.stack([_pad(table[n], maxlen=max_num_particles) for n in particle_features], axis=1)
        events, part_ft, pad_size = np.shape(x_particles)
        x_particles = np.reshape(x_particles, (events, part_ft*pad_size))
        # x_particles.type.show()
        # print('flattened')
        x_jets = np.stack([table[n] for n in jet_features], axis=1)
        # x_jets.type.show()
        chunk_X.append(np.concatenate([x_particles, x_jets], axis=1))
        chunk_y.append(np.stack([table[n] for n in labels], axis=1))
    return chunk_X, chunk_y

def _pad(a, maxlen, value=0, dtype='float32'):
    if isinstance(a, np.ndarray) and a.ndim >= 2 and a.shape[1] == maxlen:
        return a
    elif isinstance(a, ak.Array):
        if a.ndim == 1:
            a = ak.unflatten(a, 1)
        a = ak.fill_none(ak.pad_none(a, maxlen, clip=True), value)
        return ak.values_astype(a, dtype)
    else:
        x = (np.ones((len(a), maxlen)) * value).astype(dtype)
        for idx, s in enumerate(a):
            if not len(s):
                continue
            trunc = s[:maxlen].astype(dtype)
            x[idx, :len(trunc)] = trunc
        return x

def read_file(
        file_names,
        tree_name='tree',
        max_num_particles=128,
        particle_features=['part_px', 'part_py', 'part_pz', 'part_energy'],
        jet_features=['jet_pt', 'jet_eta', 'jet_phi', 'jet_energy'],
        labels=['label_QCD', 'label_Hbb', 'label_Hcc', 'label_Hgg', 'label_H4q',
                'label_Hqql', 'label_Zqq', 'label_Wqq', 'label_Tbqq', 'label_Tbl']):
    """Loads a single file from the JetClass dataset.

    **Arguments**

    - **filepath** : _str_
        - Path to the ROOT data file.
    - **max_num_particles** : _int_
        - The maximum number of particles to load for each jet. 
        Jets with fewer particles will be zero-padded, 
        and jets with more particles will be truncated.
    - **particle_features** : _List[str]_
        - A list of particle-level features to be loaded. 
        The available particle-level features are:
            - part_px
            - part_py
            - part_pz
            - part_energy
            - part_pt
            - part_eta
            - part_phi
            - part_deta: np.where(jet_eta>0, part_eta-jet_p4, -(part_eta-jet_p4))
            - part_dphi: delta_phi(part_phi, jet_phi)
            - part_d0val
            - part_d0err
            - part_dzval
            - part_dzerr
            - part_charge
            - part_isChargedHadron
            - part_isNeutralHadron
            - part_isPhoton
            - part_isElectron
            - part_isMuon
    - **jet_features** : _List[str]_
        - A list of jet-level features to be loaded. 
        The available jet-level features are:
            - jet_pt
            - jet_eta
            - jet_phi
            - jet_energy
            - jet_nparticles
            - jet_sdmass
            - jet_tau1
            - jet_tau2
            - jet_tau3
            - jet_tau4
    - **labels** : _List[str]_
        - A list of truth labels to be loaded. 
        The available label names are:
            - label_QCD
            - label_Hbb
            - label_Hcc
            - label_Hgg
            - label_H4q
            - label_Hqql
            - label_Zqq
            - label_Wqq
            - label_Tbqq
            - label_Tbl

    **Returns**

    - x_particles(_3-d numpy.ndarray_), x_jets(_2-d numpy.ndarray_), y(_2-d numpy.ndarray_)
        - `x_particles`: a zero-padded numpy array of particle-level features 
                         in the shape `(num_jets, num_particle_features, max_num_particles)`.
        - `x_jets`: a numpy array of jet-level features
                    in the shape `(num_jets, num_jet_features)`.
        - `y`: a one-hot encoded numpy array of the truth lables
               in the shape `(num_jets, num_classes)`.
    """
    # names = ROOT.std.vector('string')()
    # for n in file_names: names.push_back(n) 
    # particle_features+jet_features+labels
    
    all_chunk_X = []
    all_chunk_y = []
    with ProcessPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(chunk_processing, file, _pad, max_num_particles, particle_features, jet_features, labels ) for file in file_names]
        for future in tqdm(futures):
            (X, y) = future.result()
            all_chunk_X.extend(X)
            all_chunk_y.extend(y)

    return DataLoader(dataset=TensorDataset(
        torch.from_numpy(ak.to_numpy(np.concatenate(all_chunk_X))), 
        torch.from_numpy(ak.to_numpy(np.concatenate(all_chunk_y)))),
        batch_size=128, shuffle=True)
    # return table
    # p4 = vector.zip({'px': table['part_px'],
    #                     'py': table['part_py'],
    #                     'pz': table['part_pz'],
    #                     'energy': table['part_energy']})
    # table['part_pt'] = p4.pt
    # table['part_eta'] = p4.eta
    # table['part_phi'] = p4.phi
    # x_particles = ak.concatenate([_pad(table[n], maxlen=max_num_particles) for n in particle_features], axis=1)
    # x_particles = _pad(table[particle_features])
    # print(f'part shape: {x_particles.shape}')
    # x_particles = x_particles.reshape(x_particles.shape[0], x_particles.shape[1]*x_particles.shape[2])
    # x_particles = np.reshape(x_particles, (x_particles.shape[0], x_particles.shape[1]*x_particles.shape[2]))
    # x_particles = ak.zip([_pad(table[n], maxlen=max_num_particles) for n in particle_features])
    # x_particles = ak.flatten(ak.zip([_pad(table[n], maxlen=max_num_particles) for n in particle_features]), axis=1)
    # print('flattened')
    # x_particles.type.show()
    # # x_jets = np.stack([ak.to_numpy(table[n]).astype('float32') for n in jet_features], axis=1)
    # x_jets = ak.zip([table[n] for n in jet_features])
    # # x_jets = table[jet_features]
    # x_jets.type.show()
    # # print(x_jets.shape)
    # X = ak.zip([x_particles, x_jets])
    # # X = ak.concatenate([x_particles, x_jets], axis=1)
    # print('X Concatenated')
    # y = ak.zip([table[n]for n in labels])
    
    # # y = table[labels]
    # # return X, y
    # # return X, y
    # return DataLoader(dataset=TensorDataset(torch.Tensor(X_ak), torch.Tensor(y_ak)),
    #                   batch_size=128, shuffle=True)

/home/northnpk/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os
data_path = '/home/northnpk/Downloads/JetClass_dataset/'
train_path = [data_path + 'train/' + p for p in os.listdir(data_path + 'train')]

In [6]:
# %%cpp
# #include "TSystem.h"

# double check_mem(std::string s = ""){

#    ProcInfo_t p;
#    printf("%s - ",s.c_str());
#    gSystem->GetProcInfo(&p);
#    printf(" Rmem = %8.3f MB, Vmem = %8.f3 MB  \n",
#           p.fMemResident * 1e-3,  /// Real memory to watch for leaks                                                                                                                                                                                      
#           p.fMemVirtual  * 1e-3
#       );
#    return p.fMemResident * 1e-3;
# }

In [7]:
loader = read_file(train_path[:10])

100%|██████████| 10/10 [00:10<00:00,  1.05s/it]


In [24]:
input_shape = loader.dataset[0][0].size()[0]
output_shape = loader.dataset[0][1].size()[0]

In [25]:
len(loader)

7813

In [27]:
len(loader.dataset)

1000000

In [6]:
for idx, (x, y) in enumerate(loader):
    print(f'id{idx}) {len(x)}')

id0) 128
id1) 128
id2) 128
id3) 128
id4) 128
id5) 128
id6) 128
id7) 128
id8) 128
id9) 128
id10) 128
id11) 128
id12) 128
id13) 128
id14) 128
id15) 128
id16) 128
id17) 128
id18) 128
id19) 128
id20) 128
id21) 128
id22) 128
id23) 128
id24) 128
id25) 128
id26) 128
id27) 128
id28) 128
id29) 128
id30) 128
id31) 128
id32) 128
id33) 128
id34) 128
id35) 128
id36) 128
id37) 128
id38) 128
id39) 128
id40) 128
id41) 128
id42) 128
id43) 128
id44) 128
id45) 128
id46) 128
id47) 128
id48) 128
id49) 128
id50) 128
id51) 128
id52) 128
id53) 128
id54) 128
id55) 128
id56) 128
id57) 128
id58) 128
id59) 128
id60) 128
id61) 128
id62) 128
id63) 128
id64) 128
id65) 128
id66) 128
id67) 128
id68) 128
id69) 128
id70) 128
id71) 128
id72) 128
id73) 128
id74) 128
id75) 128
id76) 128
id77) 128
id78) 128
id79) 128
id80) 128
id81) 128
id82) 128
id83) 128
id84) 128
id85) 128
id86) 128
id87) 128
id88) 128
id89) 128
id90) 128
id91) 128
id92) 128
id93) 128
id94) 128
id95) 128
id96) 128
id97) 128
id98) 128
id99) 128
id100) 128

In [9]:
loader[1].shape

(1000000, 10)

In [12]:
count = 0
for i, (d, y) in enumerate(read_file(train_path)):
    check_x, check_y = d, y
    mem = ROOT.check_mem("gen_train")
    count+=1
print(f'Batch count: {count}')
print(f'Net mem: {mem}')

# Saving data to npz and call it later in torch dataloader

In [1]:
import numpy as np
import awkward as ak
import ROOT
from ROOT import RDataFrame
import uproot
import vector
import torch
from torch.utils.data import DataLoader, TensorDataset
from concurrent.futures import ProcessPoolExecutor
vector.register_awkward()
from tqdm.auto import tqdm
def _pad(a, maxlen, value=0, dtype='float32'):
    if isinstance(a, np.ndarray) and a.ndim >= 2 and a.shape[1] == maxlen:
        return a
    elif isinstance(a, ak.Array):
        if a.ndim == 1:
            a = ak.unflatten(a, 1)
        a = ak.fill_none(ak.pad_none(a, maxlen, clip=True), value)
        return ak.values_astype(a, dtype)
    else:
        x = (np.ones((len(a), maxlen)) * value).astype(dtype)
        for idx, s in enumerate(a):
            if not len(s):
                continue
            trunc = s[:maxlen].astype(dtype)
            x[idx, :len(trunc)] = trunc
        return x
    
def chunk_processing(file, _pad, max_num_particles, particle_features, jet_features, labels):
    chunk_x_jet = []
    chunk_x_part = []
    chunk_y = []
    for table in uproot.iterate(file, step_size=10000):
        # print('table from rdf')
        # print(table)
        p4 = vector.zip({'px': table['part_px'],
                     'py': table['part_py'],
                     'pz': table['part_pz'],
                     'energy': table['part_energy']})
        table['part_pt'] = p4.pt
        table['part_eta'] = p4.eta
        table['part_phi'] = p4.phi

        x_particles = np.stack([_pad(table[n], maxlen=max_num_particles) for n in particle_features], axis=1)
        # events, part_ft, pad_size = np.shape(x_particles)
        # x_particles = np.reshape(x_particles, (events, part_ft*pad_size))
        # x_particles.type.show()
        # print('flattened')
        x_jets = np.stack([table[n] for n in jet_features], axis=1)
        # x_jets.type.show()
        chunk_x_part.append(x_particles)
        chunk_x_jet.append(x_jets)
        chunk_y.append(np.stack([table[n] for n in labels], axis=1))
    return chunk_x_part, chunk_x_jet, chunk_y

def read_file(
        file_names,
        tree_name='tree',
        max_num_particles=128,
        input_feature=['part_px', 'part_py', 'part_pz', 'part_energy',
                       'jet_pt', 'jet_eta', 'jet_phi', 'jet_energy',
                       'label_QCD', 'label_Hbb', 'label_Hcc', 'label_Hgg', 'label_H4q',
                       'label_Hqql', 'label_Zqq', 'label_Wqq', 'label_Tbqq', 'label_Tbl'],
        particle_features=['part_pt', 'part_eta', 'part_phi', 'part_energy'],
        jet_features=['jet_pt', 'jet_eta', 'jet_phi', 'jet_energy'],
        labels=['label_QCD', 'label_Hbb', 'label_Hcc', 'label_Hgg', 'label_H4q',
                'label_Hqql', 'label_Zqq', 'label_Wqq', 'label_Tbqq', 'label_Tbl']):
    """Loads a single file from the JetClass dataset.

    **Arguments**

    - **filepath** : _str_
        - Path to the ROOT data file.
    - **max_num_particles** : _int_
        - The maximum number of particles to load for each jet. 
        Jets with fewer particles will be zero-padded, 
        and jets with more particles will be truncated.
    - **particle_features** : _List[str]_
        - A list of particle-level features to be loaded. 
        The available particle-level features are:
            - part_px
            - part_py
            - part_pz
            - part_energy
            - part_pt
            - part_eta
            - part_phi
            - part_deta: np.where(jet_eta>0, part_eta-jet_p4, -(part_eta-jet_p4))
            - part_dphi: delta_phi(part_phi, jet_phi)
            - part_d0val
            - part_d0err
            - part_dzval
            - part_dzerr
            - part_charge
            - part_isChargedHadron
            - part_isNeutralHadron
            - part_isPhoton
            - part_isElectron
            - part_isMuon
    - **jet_features** : _List[str]_
        - A list of jet-level features to be loaded. 
        The available jet-level features are:
            - jet_pt
            - jet_eta
            - jet_phi
            - jet_energy
            - jet_nparticles
            - jet_sdmass
            - jet_tau1
            - jet_tau2
            - jet_tau3
            - jet_tau4
    - **labels** : _List[str]_
        - A list of truth labels to be loaded. 
        The available label names are:
            - label_QCD
            - label_Hbb
            - label_Hcc
            - label_Hgg
            - label_H4q
            - label_Hqql
            - label_Zqq
            - label_Wqq
            - label_Tbqq
            - label_Tbl

    **Returns**

    - x_particles(_3-d numpy.ndarray_), x_jets(_2-d numpy.ndarray_), y(_2-d numpy.ndarray_)
        - `x_particles`: a zero-padded numpy array of particle-level features 
                         in the shape `(num_jets, num_particle_features, max_num_particles)`.
        - `x_jets`: a numpy array of jet-level features
                    in the shape `(num_jets, num_jet_features)`.
        - `y`: a one-hot encoded numpy array of the truth lables
               in the shape `(num_jets, num_classes)`.
    """
    # names = ROOT.std.vector('string')()
    # for n in file_names: names.push_back(n) 
    # particle_features+jet_features+labels
    input_feature = ['part_pt', 'part_eta', 'part_phi', 'part_energy']
    
    
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(chunk_processing, file, _pad, max_num_particles, particle_features, jet_features, labels ) for file in file_names]
        all_chunk_x_part=[]
        all_chunk_x_jet=[]
        all_chunk_y=[]
        for idx, future in enumerate(tqdm(futures)):
            chunk = future.result()
            all_chunk_x_part.extend(chunk[0])
            all_chunk_x_jet.extend(chunk[1])
            all_chunk_y.extend(chunk[2])
            np.savez(f'data/Jetclass_train_{idx}',
                        x_part=ak.to_numpy(np.concatenate(all_chunk_x_part)),
                        x_jet=ak.to_numpy(np.concatenate(all_chunk_x_jet)),
                        y=ak.to_numpy(np.concatenate(all_chunk_y)))
            all_chunk_x_part=[]
            all_chunk_x_jet=[]
            all_chunk_y=[]


    # return DataLoader(dataset=TensorDataset(
    #     torch.from_numpy(ak.to_numpy(np.concatenate(all_chunk_x_part))),
    #     torch.from_numpy(ak.to_numpy(np.concatenate(all_chunk_x_jet))),
    #     torch.from_numpy(ak.to_numpy(np.concatenate(all_chunk_y)))),
    #     batch_size=128, shuffle=False)

/home/northnpk/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
data_path = '/home/northnpk/Downloads/JetClass_dataset/'
train_path = [data_path + 'train/' + p for p in os.listdir(data_path + 'train')]

In [3]:
read_file(train_path[:10])

100%|██████████| 10/10 [00:11<00:00,  1.16s/it]


In [3]:
data_path = '/home/northnpk/data/'
np_list = [data_path + p for p in os.listdir(data_path)]

In [4]:
class CustomJetClassDataset(TensorDataset):
    def __init__(self, file_names):
        self.file_names = file_names
        self.file_length = [len(np.load(f)['y']) for f in file_names]
        self.total_length = sum(self.file_length)
        self.one_file_length = self.file_length[0]
        
    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        file_idx = idx//self.one_file_length
        sub_idx = idx-((file_idx)*self.file_length[file_idx])
        data = np.load(self.file_names[file_idx])
        # print(f'file_idx:{file_idx}, sub_idx:{sub_idx}')
        return torch.from_numpy(data['x_part'][sub_idx]), torch.from_numpy(data['x_jet'][sub_idx]), torch.from_numpy(data['y'][sub_idx])


In [5]:
loader = DataLoader(dataset=CustomJetClassDataset(np_list), batch_size=32)

In [6]:
for i, d in enumerate(loader):
    print(i, d)

0 [tensor([[[ 8.4252e+01,  5.8583e+01,  5.2256e+01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-3.2715e-01, -3.1567e-01, -6.5506e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.7944e+00,  2.8137e+00,  2.9184e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 8.8803e+01,  6.1526e+01,  6.3874e+01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 9.4599e+01,  6.2134e+01,  5.3795e+01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 4.6981e-01,  4.8107e-01,  2.7713e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.7692e-01,  2.8650e-01,  5.6272e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0523e+02,  6.9463e+01,  5.5874e+01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 3.5631e+02,  6.1566e+01,  4.4902e+01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 4.0342e-01,  4.1390e-01,  5.7256e-01,

KeyboardInterrupt: 